### Packages to be used

In [2]:
import pandas as pd
import numpy as np
import datetime

### Functions to be used

In [3]:
# Region of the sequence
def get_region(x):
    if len(x)>2:
        return(x[2])
    else:
        return(None)

# Week day of the sequence
def get_sunday(date_in):
    today = date_in
    next_sunday = today + datetime.timedelta(days=6-today.weekday(), weeks=0)
    return(next_sunday)

def remove_from_list(L):
    if " " in L:
        return(L.remove(" "))
    else:
        return(L)

# Dynamic feature: mean change over time
def mean_change(x):
    x = np.asarray(x)
    return (x[-1] - x[0]) / (len(x) - 1) if len(x) > 1 else np.NaN

# Mean second derivative
def mean_second_derivative_central(x):
    x = np.asarray(x)
    return (x[-1] - x[-2] - x[1] + x[0]) / (2 * (len(x) - 2)) if len(x) > 2 else np.NaN

# Relative evolution
def relative_first_derivative(x):
    t = list(x)
    first_d = [(t[i+1]-t[i])/t[i] for i in range(len(t)-1) if t[i]!= 0]
    if len(first_d) == 0:
        return([0],0)
    else:
        mean_first_d = sum(first_d)/len(first_d)
        return (first_d, mean_first_d)

# Entopy with the proportion of each variant
def entropy(x):
    v_tot = sum(x)
    prop = x/v_tot
    ent = sum([-p*np.log(p) for p in prop if p!=0])
    return(ent)

#Jaccard distance between two variants based on their respective mutations
def jaccard_distance(L1, L2):
    intersection = 0
    union = 0
    n = len(L1)
    for i in range(n):
        if (L1[i] == 1) and (L1[i] == L2[i]):
            intersection+=1
        if (L1[i] == 1) or (L2[i] == 1):
            union+=1
    j_distance = 1-intersection/union
    return(j_distance)

def absolute_distance(L1, L2):
    distance = np.absolute(np.subtract(L1,L2)).sum()
    return(distance)

def get_country(x):
    if len(x)>=2:
        return(x[1])
    else:
        return("")

In [4]:
# Getting the biological feature for the variant in its respective country
#c = country
#variants_country = sequences in country
#seq_count_per_week_pivot = weekly count of sequences per variant in country
#var = variant
#n_weeks = number of observation weeks
#threshold_substitution = threshold of the proportion of sequences for which a mutation appears to associate it with the variant
#threshold_seq = threshold of the number of sequences of a variant to consider it in a country
#country_cases = number of cases per country

def get_biological_evolution(c, variants_country,seq_count_per_week_pivot,var,n_weeks,threshold_substitution, threshold_seq, country_cases):

    #Extracting all the variants and their sequence counts
    list_variants = variants_country["Pango lineage"].unique()
    sequences_all = variants_country[variants_country["Pango lineage"] == var]
    seq_count = sequences_all.groupby("week_date")['Accession ID'].count().reset_index().rename(columns = {'Accession ID':"seq_count"})
    l_count = list(seq_count["seq_count"])
    seq_count["seq_count_2"] = l_count[1:]+[l_count[len(l_count)-1]]

    ### Significant count of sequences in the week of detection and following one
    seq_count_sig = seq_count[(seq_count["seq_count"]>=threshold_seq) & (seq_count["seq_count_2"]>=threshold_seq)]
    var_country_df = pd.DataFrame([c], columns = ["country"])
    var_country_df["variant"] = var
    if len(seq_count_sig) == 0:
        return(var_country_df)
    
    else:
        ### Selecting all sequences during observation period
        min_date = seq_count_sig["week_date"].min()
        max_date = min_date + datetime.timedelta(days=n_weeks*7)
        sequences_first = sequences_all[(sequences_all["week_date"]<= max_date) & (sequences_all["week_date"] >= min_date)]
        sequences_first['substitutions_list'] = sequences_first['AA Substitutions'].astype("str").apply(lambda x:x.replace(" ","").replace(")",",").replace("(","").split(",")[:-1])

        ### Getting the genetic mutations
        all_substitutions_list_unique = list(set(sequences_first['substitutions_list'].sum()))
        dict_substitutions = dict([[p,0] for p in all_substitutions_list_unique])
        if len(dict_substitutions) == 0:
            return(var_country_df)
        else:
            n_seq = len(sequences_first)
            for i in range(n_seq):
                sub_var = sequences_first['substitutions_list'].iloc[i]
                for s in sub_var:
                        dict_substitutions[s] += 1
            ### Getting ratio of presence of each mutation in the variant's sequences
            df_substitutions = pd.DataFrame.from_dict(dict_substitutions, orient = "index").reset_index().rename(columns = {"index":"substitutions",0:"sub_count"})
            df_substitutions["sub_ratio"] = df_substitutions["sub_count"]/n_seq
            
            ### Considering the mutations that appear with a ration higher than the threshold of substitution
            df_substitutions = df_substitutions[df_substitutions["sub_ratio"]>threshold_substitution]
            all_sub = df_substitutions["substitutions"]
            sub_ratio = df_substitutions["sub_ratio"]
            for i in range(len(all_sub)):
                var_country_df[all_sub.iloc[i]] = 1 # Could be replaced by sub_ratio.iloc[i] to get the ratio of sequences where this substitution appears

            ### Computing for the country the evolution of the proportions of each variant during the period of time when the variant appeared
            seq_count_period = seq_count_per_week_pivot[(seq_count_per_week_pivot["week_date"]<= max_date) & (seq_count_per_week_pivot["week_date"] >= min_date)].reset_index()
            seq_count_period_cases = pd.merge(seq_count_period, country_cases, on = "week_date")
            list_variants = list(list_variants)
            list_variants = [v for v in list_variants if (v!= "None" and str(v)!= 'nan')]
            seq_count_period["number_sequences"] = seq_count_period[list_variants].sum(axis = 1)

            ### Getting the dynamics of the variant and the other variants when the variant started 
            var_ratio = seq_count_period[var]/seq_count_period["number_sequences"]
            m_change_val = mean_change(var_ratio)
            m_second_derivative_central = mean_second_derivative_central(var_ratio)
            r_first_derivative_val = relative_first_derivative(var_ratio)

            r_first_derivative = r_first_derivative_val[0]
            r_first_derivative_mean = r_first_derivative_val[1]

            ### Enriching the table with variants proportions, their entropy and evolution
            list_variants_1 = [v for v in list_variants if v!= var]
            col = list_variants+["number_sequences"]
            sum_seq_over_period = seq_count_period[col].sum(axis = 0)
            if len(list_variants_1)>0:
                dominant_1 = sum_seq_over_period[list_variants_1].idxmax()
                prop_dominant_1 = sum_seq_over_period[dominant_1].max()/sum_seq_over_period["number_sequences"]
                var_country_df["dominant_1"] = dominant_1
                var_country_df["prop_dominant_1"] = prop_dominant_1

            list_variants_2 = [v for v in list_variants_1 if v!= dominant_1]
            if len(list_variants_2)>0:
                dominant_2 = sum_seq_over_period[list_variants_2].idxmax()
                prop_dominant_2 = sum_seq_over_period[dominant_2].max()/sum_seq_over_period["number_sequences"]
                var_country_df["dominant_2"] = dominant_2
                var_country_df["prop_dominant_2"] = prop_dominant_2

            var_entropy = entropy(sum_seq_over_period[list_variants].values)
            var_country_df["variants_entropy"] = var_entropy

            ratio_week = "ratio_week_"
            for i in range(len(var_ratio)):
                var_country_df[ratio_week+str(i)] = var_ratio[i]

            var_country_df["mean_ratio_change"] = m_change_val
            var_country_df["mean_second_ratio_derivative"] = m_second_derivative_central

            r_first = "r_ratio_first_derivative_"
            for i in range(len(r_first_derivative)):
                var_country_df[r_first+str(i)] = r_first_derivative[i]
            var_country_df["mean_ratio_first_derivative"] = r_first_derivative_mean

            ### Getting the dynamics of the cases and the other variants when the variant started 
            all_cases_per_million = seq_count_period_cases["new_cases_per_million"]
            cases_per_million = var_ratio*all_cases_per_million 
            total_cases = sum(cases_per_million)

            m_change_cases = mean_change(cases_per_million)
            m_second_derivative_cases = mean_second_derivative_central(cases_per_million)
            
            cases_first_derivative_val = relative_first_derivative(cases_per_million)
            cases_first_derivative_cases = cases_first_derivative_val[0]
            cases_first_derivative_cases_mean = cases_first_derivative_val[1]

            total_current_cases = all_cases_per_million.iloc[-1]

            ### Enriching the table with number of cases
            cases_week = "cases_week_"
            for i in range(len(cases_per_million)):
                var_country_df[cases_week+str(i)] = cases_per_million[i]

            var_country_df["mean_cases_change"] = m_change_cases

            cases_first = "cases_first_derivative_"
            for i in range(len(cases_first_derivative_cases)):
                var_country_df[cases_first+str(i)] = cases_first_derivative_cases[i]

            var_country_df["cases_mean_first_derivative"] = cases_first_derivative_cases_mean
            var_country_df["m_second_derivative_cases"] = m_second_derivative_cases
            var_country_df["total_current_cases"] = total_current_cases

            ### Observation period
            var_country_df["Date - start"] = min_date
            var_country_df["Date - end"] = max_date

            return(var_country_df)

### GISAID Variants data

In [5]:
### The following code takes GISAID metadata as input
path_to_gisaid_metadata = # ADD PATH TO GISAID METADATA
variants = pd.read_csv(path_to_gisaid_metadata, sep='\t')

/Users/elghalizerhouni/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
variants["Collection date"] = pd.to_datetime(variants["Collection date"])
variants["week_date"] = variants["Collection date"].apply(lambda x: get_sunday(x))

In [7]:
variants['Location_separated'] = variants['Location'].apply(lambda x: x.split(" / "))
variants['continent'] = variants['Location_separated'].apply(lambda x: x[0])
variants['country'] = variants['Location_separated'].apply(lambda x: get_country(x))
variants['region'] = variants['Location_separated'].apply(lambda x: get_region(x))
variants_unique = variants.drop_duplicates(subset = ["country", "Pango lineage"])
seq_per_country = variants.groupby("country")['Accession ID'].count().reset_index()
seq_per_country = seq_per_country.sort_values(by = 'Accession ID', ascending = False)

In [8]:
# Selecting the 30 countries of interest
list_countries_oi = ['United Kingdom', 'USA', 'Germany', 'Denmark', 'Canada', 'Japan',
       'France', 'Sweden', 'Switzerland', 'India', 'Brazil', 'Italy',
       'Spain', 'Netherlands', 'Turkey', 'Austria', 'Belgium',
        'Australia', 'Ireland', 'Mexico', 'Slovenia', 'Norway', 'Poland',
        'Israel', 'South Africa', 'Lithuania', 'Portugal', 'Finland','South Korea', 'Luxembourg']

In [9]:
n_sequences_per_country = variants.groupby("country")['Accession ID'].count().reset_index().sort_values(by ='Accession ID', ascending = False )
print("There are " + str(n_sequences_per_country.head(30)["Accession ID"].sum())+ " sequences in the countries of interest")

There are 12178888 sequences in the countries of interest


In [10]:
### Computing the number of sequences per variant every week in every country
c_sequences = variants
c_sequences['Collection date'] = pd.to_datetime(variants['Collection date'])
c_sequences_ev = c_sequences.groupby(['country','Collection date', 'Pango lineage'])['Accession ID'].count().reset_index()
c_sequences_ev = c_sequences_ev.rename(columns = {"Accession ID":"seq_count"})
c_sequences_ev = c_sequences_ev[c_sequences_ev["Collection date"]!="2020"]
c_sequences_ev["week_date"] = c_sequences_ev["Collection date"].apply(lambda x: get_sunday(x))

seq_count_per_week = c_sequences_ev.groupby(['country',"week_date","Pango lineage"])["seq_count"].sum().reset_index()
seq_count_per_week = seq_count_per_week.sort_values(by = ['country',"week_date"])

seq_count_per_week["country-week_date"] = seq_count_per_week["country"]+"//"+seq_count_per_week["week_date"].astype("str")
seq_count_per_week_pivot = seq_count_per_week.pivot(index="country-week_date", columns="Pango lineage", values="seq_count").reset_index().fillna(0)
seq_count_per_week_pivot["country"] = seq_count_per_week_pivot["country-week_date"].apply(lambda x:x.split("//")[0])
seq_count_per_week_pivot["week_date"] = seq_count_per_week_pivot["country-week_date"].apply(lambda x:x.split("//")[1])
seq_count_per_week_pivot = seq_count_per_week_pivot.drop("country-week_date", axis = 1)
seq_count_per_week_pivot["week_date"] = pd.to_datetime(seq_count_per_week_pivot["week_date"])

var_col = seq_count_per_week_pivot.columns[:-2]
seq_count_per_week_pivot["total_seq"] = seq_count_per_week_pivot[var_col].sum(axis = 1)

### Median number of variants for the 30 most reporting country.

In [11]:
n_variant_per_country = variants_unique.groupby("country")["Pango lineage"].count().reset_index()

In [12]:
n_variants = len(variants_unique["Pango lineage"].unique())
print("There are "+ str(n_variants)+" in the world.")

There are 1878 in the world.


In [13]:
top_30_countries = ['United Kingdom', 'USA', 'Germany', 'Denmark', 'Canada', 'Japan',
       'France', 'Sweden', 'Switzerland', 'India', 'Brazil', 'Italy',
       'Spain', 'Netherlands', 'Turkey', 'Austria', 'Belgium',
        'Australia', 'Ireland', 'Mexico', 'Slovenia', 'Norway', 'Poland',
        'Israel', 'South Africa', 'Lithuania', 'Portugal', 'Finland','South Korea', 'Luxembourg']
top_30_countries = pd.DataFrame(top_30_countries, columns = ["country"])

In [14]:
n_variant_top_30 = pd.merge(n_variant_per_country, top_30_countries, on = "country")

In [15]:
median = n_variant_top_30['Pango lineage'].median()
print("There is a median of "+ str(median)+" in top 30 most reporting countries.")

There is a median of 490.0 in top 30 most reporting countries.


### Covid cases data

In [16]:
### The following code take Our World in Data COVID-19 as input https://ourworldindata.org/coronavirus
owid_path = #ADD PATH TO Our World in Data COVID-19 data
covid_data = pd.read_csv(owid_path) 
covid_data['new_cases'] = abs(covid_data['new_cases'])
covid_data['new_cases_per_million'] = abs(covid_data['new_cases_per_million'])
covid_data["date"] = pd.to_datetime(covid_data["date"])
covid_data["week_date"] = covid_data["date"].apply(lambda x: get_sunday(x))
covid_data = covid_data.fillna(0)
global_cases = covid_data.groupby(["week_date", "location"])['new_cases_per_million'].sum().reset_index()

In [17]:
global_cases["country"] = global_cases["location"].replace("United States", "USA")
global_cases["country"] = global_cases["country"].replace('Czechia', "Czech Republic")
global_cases = global_cases[["week_date","country",'new_cases_per_million']]

In [18]:
seq_count_per_week_cases_pivot = pd.merge(seq_count_per_week_pivot, global_cases, on = ["week_date","country"])

### Summary statistics

In [19]:
### Number of variants
n_variants = len(variants['Pango lineage'].unique())
print("There are " + str(n_variants) + " variants following the Pango lineage")

There are 1878 variants following the Pango lineage


In [20]:
### Number of variants per country
unique_variants_per_country = variants.drop_duplicates(['country','Pango lineage'])
n_variants_per_country = unique_variants_per_country.groupby("country")['Pango lineage'].count().reset_index()

In [21]:
n_seq_country = variants.groupby("country")['Accession ID'].count().reset_index()

In [22]:
n_locations = len(n_variants_per_country["country"].unique())
print("There are " + str(n_locations) + " locations")

There are 220 locations


In [23]:
median_variants = n_variants_per_country["Pango lineage"].median()
print("There is a median of " + str(median_variants) + " variants per location")

There is a median of 65.5 variants per location


### Parameters to compute independent variables

In [24]:
n_weeks = 2 #Indicate the number of weeks of the observation period
threshold_substitution = 0.10 #Some mutations don't appear in all sequences of the same variant. This threshold indicate the minimum proportion of sequences that a mutation needs to be identified in to consider it
threshold_seq = 5 #Indicate the minimum number of sequences of variant to consider it during a week

### Variants biological evolution features

In [1]:
### Getting the biological features for every variant in every country, starting by the United Kingdom
country = "United Kingdom"
variants_country = variants[(variants["country"] == country) & (variants["week_date"]>"2020-02-01")]
c_sequences = variants_country[variants_country["Collection date"]!="2020"]
seq_count_per_week = c_sequences.groupby(["week_date","Pango lineage"])['Accession ID'].count().reset_index()
seq_count_per_week = seq_count_per_week.rename(columns = {"Accession ID":"seq_count"})
seq_count_per_week = seq_count_per_week.sort_values(by = ["week_date"])
seq_count_per_week_pivot = seq_count_per_week.pivot(index="week_date", columns="Pango lineage", values="seq_count").reset_index().fillna(0)
variants_list = list(variants_country["Pango lineage"].dropna().unique())
country_cases = global_cases[global_cases["country"] == country]

var = variants_list[0]
c = country
biological_evolution_country = get_biological_evolution(c, variants_country,seq_count_per_week_pivot,var,n_weeks,threshold_substitution, threshold_seq, country_cases)
for var in variants_list[1:]:
    biological_evolution = get_biological_evolution(c, variants_country,seq_count_per_week_pivot,var,n_weeks,threshold_substitution, threshold_seq, country_cases)
    biological_evolution_country = pd.concat([biological_evolution_country,biological_evolution], axis = 0)

biological_evolution_country = biological_evolution_country[biological_evolution_country["Date - start"].isna() == False].fillna(0)

In [2]:
countries_list = [c for c in list_countries_oi if c != "United Kingdom"]

for c in countries_list:
    print(c)
    variants_country = variants[(variants["country"] == c) & (variants["week_date"]>"2020-02-01")]
    c_sequences = variants_country[variants_country["Collection date"]!="2020"]
    seq_count_per_week = c_sequences.groupby(["week_date","Pango lineage"])['Accession ID'].count().reset_index()
    seq_count_per_week = seq_count_per_week.rename(columns = {"Accession ID":"seq_count"})
    seq_count_per_week = seq_count_per_week.sort_values(by = ["week_date"])
    seq_count_per_week_pivot = seq_count_per_week.pivot(index="week_date", columns="Pango lineage", values="seq_count").reset_index().fillna(0)
    variants_list = list(variants_country["Pango lineage"].dropna().unique())
    variants_list = [v for v in variants_list if v!= "None"]
    
    country_cases = global_cases[global_cases["country"] == c]
    if len(country_cases)>0:
        var = variants_list[0]
        biological_evolution_loc = get_biological_evolution(c, variants_country,seq_count_per_week_pivot,var,n_weeks,threshold_substitution, threshold_seq, country_cases)

        for var in variants_list[1:]:
            biological_evolution = get_biological_evolution(c, variants_country,seq_count_per_week_pivot,var,n_weeks,threshold_substitution, threshold_seq, country_cases)
            biological_evolution = biological_evolution.fillna(0)
            biological_evolution_loc = pd.concat([biological_evolution_loc,biological_evolution], axis = 0)

        biological_evolution_country = pd.concat([biological_evolution_country,biological_evolution_loc], axis = 0)

In [27]:
biological_evolution_all = biological_evolution_country
biological_evolution_all = biological_evolution_all[biological_evolution_all["Date - end"].isna() == False]
biological_evolution_all = biological_evolution_all.fillna(0)
biological_evolution_all.to_csv("generated_data/biological_evolution_country_"+str(n_weeks)+"weeks.csv")

In [28]:
biological_evolution_all = pd.read_csv("generated_data/biological_evolution_country_"+str(n_weeks)+"weeks.csv").drop("Unnamed: 0", axis = 1)

### Calculation of the number of mutations per viral protein

In [29]:
### Isolating biological features
list_var_col = ['variant','cases_first_derivative_0',
       'cases_first_derivative_1', 'cases_first_derivative_2','cases_mean_first_derivative', 'cases_week_0', 'cases_week_1','cases_week_2', 'cases_week_3', 'country', 'dominant_1', 'dominant_2','m_second_derivative_cases', 'mean_cases_change', 'mean_ratio_change',
       'mean_ratio_first_derivative', 'mean_second_ratio_derivative','prop_dominant_1', 'prop_dominant_2', 'r_ratio_first_derivative_0','r_ratio_first_derivative_1', 'r_ratio_first_derivative_2', 'ratio_week_0', 'ratio_week_1', 'ratio_week_2', 'ratio_week_3', 'total_current_cases', 'variants_entropy', 'number_of_mutation', 'difference_from_dom',   'current_new_cases_per_million',
       'current_total_vaccinations_per_hundred', 'current_stringency_index',
        "max_week_cases", "n_variants_past","cancel_public_events",'restriction_gatherings','Date - start', 'Date - end', '']
all_col = biological_evolution_all.columns
bio_col = [c for c in all_col if (c not in list_var_col)]

In [30]:
### Getting the number of mutations per protein

### Non-structural protein
NSP_list = []
### N protein
N_list = []
### Spike protein
Spike_list = []
### M protein
M_list = []
### E protein
E_list = []

other_list = []

for col in bio_col:
    if col[0:3] == 'NSP' or col[0:2] == 'NS':
        NSP_list+=[col]
    elif col[0:5] == 'Spike':
        Spike_list += [col]
    elif col[0] == 'N':
        N_list += [col]
    elif col[0] == 'M':
        M_list += [col]
    elif col[0] == 'E':
        E_list += [col]
        
biological_evolution_all["total_number_of_mutations"] = biological_evolution_all[bio_col].sum(axis = 1)
biological_evolution_all["NSP_mutations"] = biological_evolution_all[NSP_list].sum(axis = 1)
biological_evolution_all["Spike_mutations"] = biological_evolution_all[Spike_list].sum(axis = 1)
biological_evolution_all["N_mutations"] = biological_evolution_all[N_list].sum(axis = 1)
biological_evolution_all["M_mutations"] = biological_evolution_all[M_list].sum(axis = 1)
biological_evolution_all["E_mutations"] = biological_evolution_all[E_list].sum(axis = 1)

## Computing Jaccard and absolute distance between variant and the dominant variant during the observation period

In [3]:
### Computing Jaccard and absolute distance between variant and the dominant variant during the observation period

country_list = list(biological_evolution_all["country"].unique())
c = country_list[0]
country_variants = biological_evolution_all[biological_evolution_all["country"] == c]
n_var = len(country_variants)
absolute_distance_list = []
jaccard_distance_list = []
for i in range(n_var):
    var_data = country_variants.iloc[i]
    dom = var_data["dominant_1"]
    dom_data = country_variants[country_variants["variant"] == dom]
    bio_var_1 = list(np.array(var_data[bio_col])[1:])
    bio_var_2 = list(np.array(dom_data[bio_col])[0][1:])
    absolute_distance_v = absolute_distance(bio_var_1, bio_var_2)
    jaccard_distance_v = jaccard_distance(bio_var_1, bio_var_2)
    absolute_distance_list +=[absolute_distance_v]
    jaccard_distance_list +=[jaccard_distance_v]

country_variants["absolute_distance"] = absolute_distance_list
country_variants["jaccard_distance"] = jaccard_distance_list

for c in country_list[1:]:
    country_variants_loc = biological_evolution_all[biological_evolution_all["country"] == c]
    n_var = len(country_variants_loc)
    absolute_distance_list = []
    jaccard_distance_list = []
    for i in range(n_var):
        var_data = country_variants_loc.iloc[i]
        dom = var_data["dominant_1"]
        dom_data = country_variants_loc[country_variants_loc["variant"] == dom]
        if len(dom_data)>0:
            bio_var_1 = list(np.array(var_data[bio_col])[1:])
            bio_var_2 = list(np.array(dom_data[bio_col])[0][1:])
            absolute_distance_v = absolute_distance(bio_var_1, bio_var_2)
            jaccard_distance_v = jaccard_distance(bio_var_1, bio_var_2)
            absolute_distance_list +=[absolute_distance_v]
            jaccard_distance_list +=[jaccard_distance_v]
        else:
            absolute_distance_list +=[0]
            jaccard_distance_list +=[0]
    country_variants_loc["absolute_distance"] = absolute_distance_list
    country_variants_loc["jaccard_distance"] = jaccard_distance_list
    
    country_variants = pd.concat([country_variants,country_variants_loc], axis = 0)

In [32]:
country_variants.to_csv("generated_data/country_variants"+str(n_weeks)+"weeks.csv")

### Computing the heterogeneity during observation period of each variant

In [33]:
country_variants["Date - start"] = pd.to_datetime(country_variants["Date - start"])
country_variants["Date - end"] = pd.to_datetime(country_variants["Date - end"])

In [35]:
### Country variants
country_list = list(country_variants["country"].unique())
heterogeneity_country_all = []

for country in country_list:
    country_variant_data = country_variants[country_variants["country"] == country]
    bio_col = country_variant_data.columns[2:-9]

    ### Country sequences
    seq_country = seq_count_per_week_cases_pivot[seq_count_per_week_cases_pivot["country"]== country] #.head(10)

    ### Hetereogeneity measure for each variant during observation period
    heterogeneity_country = []
    n_country_var = len(country_variant_data)

    for k in range(n_country_var):
        country_var = country_variant_data.iloc[k]
        date_start = country_var["Date - start"] + datetime.timedelta(days=-7)
        date_end = country_var["Date - end"]
        seq_country_period = seq_country[(seq_country["week_date"] >= date_start) & (seq_country["week_date"] <= date_end)]
        var_col = seq_country.columns[:-4]
        seq_per_var = seq_country_period[var_col].sum(axis = 0).sort_values(ascending = False)
        total_seq = seq_country_period["total_seq"].sum(axis = 0)
        ratio_per_var = seq_per_var/total_seq 
        ratio_per_var_df = pd.DataFrame(ratio_per_var.head(10)).reset_index().rename(columns = {0:"ratio"})
        variants_index = ratio_per_var_df["index"]
        ratios = ratio_per_var_df["ratio"]
        variants_list = list(country_variant_data["variant"])

        variants_in = []
        ratios_in = []
        for k in range(10):
            if variants_index[k] in variants_list:
                variants_in += [variants_index[k]]
                ratios_in += [ratios[k]]

        n_var = len(variants_in)
        heterogeneity = 0
        R = 0

        for i in range(n_var):
            for j in range(n_var):
                var_1 = variants_in[i]
                r_1 = ratios_in[i]
                var_2 = variants_in[j]
                r_2 = ratios_in[j]

                var_info_1 = country_variant_data[(country_variant_data["variant"] == var_1)][bio_col].values[0]
                var_info_2 = country_variant_data[(country_variant_data["variant"] == var_2)][bio_col].values[0]

                j_d = jaccard_distance(var_info_1,var_info_2)
                heterogeneity += r_1*r_2*j_d
                R += r_1*r_2

        heterogeneity_country += [heterogeneity]   
    heterogeneity_country_all += heterogeneity_country

In [ ]:
country_variants["heterogeneity"] = heterogeneity_country_all
country_variants.to_csv("generated_data/country_variants"+str(n_weeks)+"weeks.csv")

### Epidemiological evolution and restriction features during obervation period

In [ ]:
country_variants = country_variants.drop_duplicates(subset = ["country", "variant", "Date - start", "Date - end"])
country_variants["Date - end"] = pd.to_datetime(country_variants["Date - end"])
country_variants["Date - start"] = pd.to_datetime(country_variants["Date - start"])
country_variants["time_period"] = (country_variants["Date - end"] - country_variants["Date - start"]).dt.days
country_variants = country_variants[country_variants["time_period"] == n_weeks*7]

weekly_cases = covid_data.groupby(["week_date", "location"])['new_cases_per_million'].sum().reset_index()
weekly_mean_total_vaccinations = covid_data.groupby(["week_date", "location"])['total_vaccinations_per_hundred'].mean().reset_index()
weekly_stringency = covid_data.groupby(["week_date", "location"])["stringency_index"].mean().reset_index()
case_vacc = pd.merge(weekly_cases,weekly_mean_total_vaccinations, on =  ["week_date", "location"])
case_vacc_stringency = pd.merge(case_vacc,weekly_stringency, on =  ["week_date", "location"]).rename(columns = {"location":"country"})
case_vacc_stringency["week_date"] = pd.to_datetime(case_vacc_stringency["week_date"])
case_vacc_stringency["country"] = case_vacc_stringency["country"].replace("United States","USA")

epidemiology_situation = pd.merge(country_variants[["variant", "country", "Date - end"]], case_vacc_stringency, left_on = ["Date - end", "country"], right_on = ["week_date", "country"], how = "left")

col = ['variant', 'country', 'Date - end',  'new_cases_per_million', 'total_vaccinations_per_hundred','stringency_index']

epidemiology_situation = epidemiology_situation[col].rename(columns = {"new_cases_per_million":"current_new_cases_per_million", "total_vaccinations_per_hundred":"current_total_vaccinations_per_hundred", "stringency_index":"current_stringency_index"})

country_variants_epi = pd.merge(country_variants, epidemiology_situation, on = ['variant', 'country', 'Date - end'], how = "inner")

all_indep_variables_oi = country_variants_epi[["country", "variant", "Date - end"]]

### Propagration in other countries 

In [ ]:
data = country_variants_epi

In [ ]:
last_cases_l = ["cases_week_1", "cases_week_2","cases_week_3","cases_week_4"]
last_cases = last_cases_l[n_weeks-1]

In [ ]:
n_countries_var = len(data)
max_week_cases_l = []
n_countries_l = []
max_cumulative_country_l = []
max_cumulative_mean_country_l = []

for i in range(n_countries_var):
    data_loc = data.iloc[i]
    var = data_loc["variant"]
    country = data_loc["country"]
    date_end = data_loc["Date - end"]

    var_in_other_countries = data[(data["variant"] == var) & (data["Date - start"] <= date_end) & (data["country"] != country)]
    countries_oi = var_in_other_countries[["Date - start","country","variant",last_cases]]["country"]
    other_countries_seq = pd.merge(seq_count_per_week_cases_pivot,countries_oi, on = "country")
    other_countries_seq_oi = other_countries_seq[[var, "country", "week_date", "total_seq", "new_cases_per_million"]]
    other_countries_seq_oi = other_countries_seq_oi[other_countries_seq_oi["week_date"] <= date_end]
    other_countries_seq_oi["ratio_var"] = other_countries_seq_oi[var]/other_countries_seq_oi["total_seq"]
    other_countries_seq_oi["cases_var"] = other_countries_seq_oi["ratio_var"]*other_countries_seq_oi["new_cases_per_million"]
    
    cumulative_country = other_countries_seq_oi.groupby("country")["cases_var"].sum().reset_index()
    mean_country = other_countries_seq_oi.groupby("country")["cases_var"].mean().reset_index()
    
    max_cumulative_mean_country = mean_country["cases_var"].max()
    max_cumulative_country = cumulative_country["cases_var"].max()
    max_week_cases = other_countries_seq_oi["cases_var"].max()
    n_countries = len(countries_oi)

    max_cumulative_mean_country_l += [max_cumulative_mean_country]
    max_week_cases_l += [max_week_cases]
    n_countries_l += [n_countries]
    max_cumulative_country_l += [max_cumulative_country]
    
data["max_cumulative_mean_country"] = max_cumulative_mean_country_l
data["max_week_cases"] = max_week_cases_l
data["max_cumulative_country"] = max_cumulative_country_l
data["n_countries"] = n_countries_l

In [ ]:
data.to_csv("generated_data/data_indep_other_countries_"+str(n_weeks)+"_weeks.csv")

In [ ]:
country_l = list(data["country"].unique())
n_variants_l = []
for c in country_l:
    data_country = data[data["country"] == c]
    n_country = len(data_country)
    for i in range(n_country):
        data_country_loc = data_country.iloc[i]
        var = data_country_loc["variant"]
        date_end = data_country_loc["Date - end"]
        n_variants = len(data_country[data_country["Date - start"] <= date_end])
        n_variants_l +=[n_variants]
data["n_variants_past"] = n_variants_l

In [ ]:
data.to_csv("generated_data/data_indep_other_countries_"+str(n_weeks)+"_weeks.csv")

### Social restriction data

In [ ]:
# Restrictions on public events
mobility_data_path = #PATH TO MOBILITY DATA FROM OUR WORLD IN DATA
public_events = pd.read_csv(mobility_data_path)
public_events = public_events.rename(columns = {"Day":"date", "Entity":"country"})
public_events["date"] = pd.to_datetime(public_events["date"])
public_events["week_date"] = public_events["date"].apply(lambda x: get_sunday(x))
public_events["country"] = public_events["country"].replace("United States", "USA")
public_events_country = public_events.groupby(["country", "week_date"])['cancel_public_events'].mean().reset_index()
data_public = pd.merge(data,public_events_country, left_on = ["country", "Date - end"], right_on = ["country", "week_date"])

In [ ]:
# Restrictions on social gathering
gathering_data_path = #PATH TO PUBLIC GATHERING DATA FROM OUR WORLD IN DATA
social_gathering = pd.read_csv(gathering_data_path)
social_gathering = social_gathering.rename(columns = {"Day":"date", "Entity":"country"})
social_gathering["date"] = pd.to_datetime(social_gathering["date"])
social_gathering["country"] = social_gathering["country"].replace("United States", "USA")
social_gathering["week_date"] = social_gathering["date"].apply(lambda x: get_sunday(x))
social_gathering_country = social_gathering.groupby(["country", "week_date"])["restriction_gatherings"].mean().reset_index()
data_public_social_d = pd.merge(data_public,social_gathering_country, left_on = ["country", "Date - end"], right_on = ["country", "week_date"])

In [ ]:
data_public_social_d.to_csv("generated_data/data_public_social_d_"+str(n_weeks)+".csv")

### Vaccinations data

In [ ]:
weekly_mean_total_vaccinations = covid_data.groupby(["week_date", "location"])['people_vaccinated_per_hundred'].mean().reset_index().rename(columns = {"location":"country"})
weekly_mean_total_vaccinations["country"] = weekly_mean_total_vaccinations["country"].replace("United States","USA")

epidemiology_situation = pd.merge(data_public_social_d[["variant", "country", "Date - end"]], weekly_mean_total_vaccinations, left_on = ["Date - end", "country"], right_on = ["week_date", "country"], how = "left")

col = ['variant', 'country', 'Date - end', 'people_vaccinated_per_hundred']

epidemiology_situation = epidemiology_situation[col].rename(columns = {"new_cases_per_million":"current_new_cases_per_million", "total_vaccinations_per_hundred":"current_total_vaccinations_per_hundred"})

data_public_social_d_epi = pd.merge(data_public_social_d, epidemiology_situation, on = ['variant', 'country', 'Date - end'], how = "inner")

In [ ]:
data_public_social_d_epi.to_csv("generated_data/data_public_social_timing_vax_"+str(n_weeks)+".csv")

### Adding dependent variable

In [ ]:
### The dependent variable is the number of cases that each variant in each country in the next 1 month, 2 months, 3 months
def get_dep_var(c):
    variants_country = variants[(variants["country"] == c) & (variants["week_date"]>"2020-02-01")]
    c_sequences = variants_country[variants_country["Collection date"]!="2020"]
    seq_count_per_week = c_sequences.groupby(["week_date","Pango lineage"])['Accession ID'].count().reset_index()
    seq_count_per_week = seq_count_per_week.rename(columns = {"Accession ID":"seq_count"})
    seq_count_per_week = seq_count_per_week.sort_values(by = ["week_date"])
    seq_count_per_week_pivot = seq_count_per_week.pivot(index="week_date", columns="Pango lineage", values="seq_count").reset_index().fillna(0)
    variants_list = list(variants_country["Pango lineage"].dropna().unique())
    variants_list = [v for v in variants_list if v!= "None"]

    country_cases = global_cases[global_cases["country"] == c]

    bio_col = seq_count_per_week_pivot.columns[2:]
    seq_count_per_week_pivot["tot_seq"] = seq_count_per_week_pivot[bio_col].sum(axis = 1)
    seq_cases = pd.merge(seq_count_per_week_pivot,country_cases, on = ["week_date"])

    dep_var = []
    all_indep_variables_country = all_indep_variables_oi[all_indep_variables_oi["country"] == c]
    n_var_c = len(all_indep_variables_country)
    for i in range(n_var_c):
        var_country = all_indep_variables_country.iloc[i]
        var = var_country["variant"]
        date_end = var_country["Date - end"]
        
        ### Assessment after 3 months
        date_month_3 = date_end+datetime.timedelta(days=3*28)
        seq_count_period_3 = seq_cases[(seq_cases["week_date"]>= date_end) & (seq_cases["week_date"]<= date_month_3)]
        seq_count_period_3_sum = seq_count_period_3.sum(axis = 0)
        ratio_month_3 = seq_count_period_3_sum[var]/seq_count_period_3_sum["tot_seq"]
        count_month_3 = ratio_month_3*seq_count_period_3_sum["new_cases_per_million"]

        ### Assessment after 2 months
        date_month_2 = date_end+datetime.timedelta(days=2*28)
        seq_count_period_2 = seq_cases[(seq_cases["week_date"]>= date_end) & (seq_cases["week_date"]<= date_month_2)]
        seq_count_period_2_sum = seq_count_period_2.sum(axis = 0)
        ratio_month_2 = seq_count_period_2_sum[var]/seq_count_period_2_sum["tot_seq"]
        count_month_2 = ratio_month_2*seq_count_period_2_sum["new_cases_per_million"]

        ### Assessment after 1 month
        date_month_1 = date_end+datetime.timedelta(days=28)
        seq_count_period_1 = seq_cases[(seq_cases["week_date"]>= date_end) & (seq_cases["week_date"]<= date_month_1)]
        seq_count_period_1_sum = seq_count_period_1.sum(axis = 0)
        ratio_month_1 = seq_count_period_1_sum[var]/seq_count_period_1_sum["tot_seq"]
        count_month_1 = ratio_month_1*seq_count_period_1_sum["new_cases_per_million"]
        
        dep_var += [[c, var, date_end, ratio_month_3, count_month_3, ratio_month_2, count_month_2, ratio_month_1, count_month_1]]
        
    dep_var_df = pd.DataFrame(dep_var, columns = ["country","var","date_end","ratio_month_3", "count_month_3", "ratio_month_2", "count_month_2", "ratio_month_1", "count_month_1"])
    return(dep_var_df)

In [ ]:
country_list = list(all_indep_variables_oi["country"].unique())
c = country_list[0]
dep_var_df_global = get_dep_var(c)

for c in country_list[1:]:
    dep_var_df = get_dep_var(c)
    dep_var_df_global = pd.concat([dep_var_df_global,dep_var_df], axis = 0)

dep_var_df_global.to_csv("generated_data/dep_var_df_global_"+str(n_weeks)+".csv")

In [ ]:
### Merging the dependent and independent variables
dep_var_df_global = dep_var_df_global.rename(columns = {"var":"variant","date_end":"Date - end"})
dep_var_df_global["Date - end"] = pd.to_datetime(dep_var_df_global["Date - end"])
data_indep_dep = pd.merge(data_public_social_d_epi, dep_var_df_global, on = ["variant", "country", "Date - end" ])
data_indep_dep.to_csv("generated_data/data_indep_dep_"+str(n_weeks)+"_weeks.csv")